# Beginning of First Part

In [76]:
import re
import os
import bcolz
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
import _pickle as cPickle
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers import Dense, Dropout, Activation, Flatten
from numpy.random import random, permutation, randn, normal, uniform, choice
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional

In [51]:
# path = get_file(fname = 'allPyPrograms_no_comments.txt', origin = 'file:///homw/wtemp/585/allPyPrograms_no_comments.txt')
path = os.getcwd() + '/allPyPrograms_no_comments.txt'
text_all = open(path).read()
text = text_all[:1000000]
print(len(text))

1000000


In [52]:
from pprint import pprint
pprint(text[:300])

('def sort_blocks():\n'
 "    with open('README.md', 'r') as read_me_file:\n"
 '        read_me = read_me_file.read()\n'
 "    table_of_contents = ''.join(read_me.split('- - -')[0])\n"
 "    blocks = ''.join(read_me.split('- - -')[1]).split('\\n# ')\n"
 '    for i in range(len(blocks)):\n'
 '        if i == 0:\n'
 '            blocks[i]')


In [53]:
chars = sorted(list(set(text)))
print(len(chars)+1)

247


In [54]:
chars.insert(0, '\0')

In [55]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [56]:
total_index = [char_to_index[char] for char in text]

In [57]:
total_index[:10]

[70, 71, 72, 2, 85, 81, 84, 86, 65, 68]

In [58]:
''.join(index_to_char[i] for i in total_index[:25])

'def sort_blocks():\n    wi'

In [59]:
pred_num = 3
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [60]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [61]:
pprint(X)

[array([70,  2, 84, ...,  2,  2, 75]),
 array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89])]


In [62]:
Y[:8]

array([ 2, 84, 68, 69, 10,  1,  2, 75])

In [63]:
X[0].shape, Y.shape

((333330,), (333330,))

In [64]:
hidden_layers = 256
vocab_size = len(chars)+1
n_fac = 42

In [65]:
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, activation='relu'),
        Dense(vocab_size, activation='softmax')
    ])

In [66]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 3, 42)             10416     
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 256)               76544     
_________________________________________________________________
dense_3 (Dense)              (None, 248)               63736     
Total params: 150,696
Trainable params: 150,696
Non-trainable params: 0
_________________________________________________________________


In [67]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [78]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=25)

Epoch 1/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3134
Epoch 2/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3115
Epoch 3/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3090
Epoch 4/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3070
Epoch 5/25
333330/333330 [==============================] - 108s 324us/step - loss: 1.3044
Epoch 6/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3033
Epoch 7/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.3008
Epoch 8/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.2989
Epoch 9/25
333330/333330 [==============================] - 107s 322us/step - loss: 1.2970
Epoch 10/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.2947
Epoch 11/25
333330/333330 [==============================] - 108s 323us/step - loss: 1.29

In [20]:
model.save_weights('simpleRNN_3pred.h5')

In [21]:
model.load_weights('simpleRNN_3pred.h5')

In [22]:
model.save_weights('simpleRNN_7pred.h5')

In [23]:
model.load_weights('simpleRNN_7pred.h5')

In [69]:
def predict_next_char(inp):
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [70]:
predict_next_char(' de')

'f'

In [71]:
predict_next_char('t34')

'8'

In [72]:
prog = "while True:"
for i in range(1000):
    prog = prog + predict_next_char(prog[-3:]).replace('\n', ' ')
prog

"while True: thefuck.rules.get_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_command('git in return self._downloader.pat_new_comma

In [73]:
def recom_model1(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        seen = observed_seq + word
        p = predict_next_char(seen[-3:])
        word = word + p
        last_item = p
    return word

In [74]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model1)

dummy.py  has no hole to predict
(b'', b'')
(b'', b'')


(70, 17, 2)

# Beginning of Second Part

In [28]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [29]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [30]:
X

[array([70,  2, 84, ...,  2,  2, 75]),
 array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89])]

In [31]:
Y_return

[array([71, 85, 86, ...,  2,  9, 71]),
 array([72, 81, 65, ...,  2, 88, 89]),
 array([ 2, 84, 68, ...,  2, 75, 65])]

In [32]:
vocab_size = 247
n_fac = 42
hidden_layers = 256

In [33]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [34]:
return_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 3, 42)             10374     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 3, 256)            76544     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 247)            63479     
Total params: 150,397
Trainable params: 150,397
Non-trainable params: 0
_________________________________________________________________


In [35]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [36]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [37]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
333330/333330 [==============================] - 68s 204us/step - loss: 2.3099
Epoch 2/5
333330/333330 [==============================] - 63s 190us/step - loss: 2.0601
Epoch 3/5
333330/333330 [==============================] - 72s 217us/step - loss: 2.0093
Epoch 4/5
333330/333330 [==============================] - 72s 217us/step - loss: 1.9826
Epoch 5/5
333330/333330 [==============================] - 73s 218us/step - loss: 1.9645


In [38]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
333330/333330 [==============================] - 72s 215us/step - loss: 1.9513
Epoch 2/5
333330/333330 [==============================] - 63s 188us/step - loss: 1.9415
Epoch 3/5
333330/333330 [==============================] - 62s 187us/step - loss: 1.9334
Epoch 4/5
333330/333330 [==============================] - 63s 188us/step - loss: 1.9269
Epoch 5/5
333330/333330 [==============================] - 65s 195us/step - loss: 1.9212


In [39]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

Epoch 1/5
333330/333330 [==============================] - 63s 190us/step - loss: 1.9165
Epoch 2/5
333330/333330 [==============================] - 63s 188us/step - loss: 1.9121
Epoch 3/5
333330/333330 [==============================] - 66s 199us/step - loss: 1.9087
Epoch 4/5
333330/333330 [==============================] - 62s 187us/step - loss: 1.9053
Epoch 5/5
333330/333330 [==============================] - 62s 186us/step - loss: 1.9026


In [40]:
return_model.save_weights('return_sequences_25.h5')

In [41]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [42]:
predict_every_char('qor item i')

'udt n rpini '